In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
import json
from datetime import datetime
from collections import defaultdict
import sys
sys.path.append("..")

import jax

from federated_library.distributions import convert_to_federated_data
from federated_library.dataset_loader import load_tf_dataset
from federated_library.train_fed_avg import fed_avg_gridsearch
from heuristic.read_data import get_local_res, get_federated_val_acc
from heuristic.heuristic_funcs import aggregate_results
from constants import SKEWS, DATASETS, NR_PARTIES

HEUR_VERSIONS = [1, 2, 3]

2022-02-07 23:15:21.461227: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


In [ ]:
# Evaluate heuristic performance

# NR_EVALUATIONS = DATASETS x SKEW_TYPES x NR_CLIENTS x SKEW

for dataset_name in DATASETS:
    for skew_type in SKEWS.keys():
        print(f"DATASET: {dataset_name}, SKEW_TYPE: {skew_type}")
        ds, (x_test, y_test), ds_info = load_tf_dataset(
            dataset_name=dataset_name,
            skew_type=skew_type,
            decentralized=False,
            display=False
        )

        test_split = convert_to_federated_data(
            x_test, y_test, ds_info, is_train=False)

        hp_configs = dict(
            act_fn=[jax.nn.relu],
            client_lr=[0.01],
            client_momentum=[0.01],
            epochs_per_round=[2],
            rounds=[30],
            runs=[1]
        )

        heur_results = defaultdict(list)

        for nr_clients in NR_PARTIES:
            ds_info['num_clients'] = nr_clients
            for skew in SKEWS[skew_type]:
                print(f"{nr_clients} clients, skew={skew}")
                hp_configs['skew'] = [skew]

                hps, accs, best_acc, ratios = get_local_res(
                    dataset_name, skew, nr_clients, skew_type
                )

                agg_hp_configs = {
                    v: aggregate_results(
                        hps, accs, ratios,
                        type_of_skew=skew_type, v=v) for v in HEUR_VERSIONS
                }

                for v in HEUR_VERSIONS:
                    for hp, value in agg_hp_configs[v].items():
                        hp_configs[hp] = [value]

                    heur_acc, hp_setting = fed_avg_gridsearch(
                        hp_configs, ds, test_split, ds_info, display=False
                    )[0]

                    fedavg_acc = get_federated_val_acc(
                        dataset_name, skew, nr_clients, skew_type)

                    heur_results[v].append(dict({
                        **hp_setting,
                        **{"heur_acc": heur_acc,
                           "fedavg_acc": fedavg_acc,
                           "dataset": dataset_name,
                           "nr_clients": nr_clients,
                           "skew_type": skew_type
                           }
                    }))

                    print(f"v{v}", heur_results[v][-1])

    if not os.path.exists(f"heur_results/"):
        os.mkdir(f"heur_results/")

    file_name = (f"heur_results/heur_{dataset_name}_{datetime.now()}.json")

    # Save heuristic results
    with open(file_name, 'w') as outfile:
        json.dump(heur_results, outfile)